In [2]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.4 MB/s eta

In [3]:
from google.colab import userdata
llm=ChatGroq(temperature=0,
             model_name="llama3-70b-8192",
             api_key=userdata.get('GROQ_API_KEY'))

In [4]:
JuniorLegalExpert = Agent(
    llm=llm,
    role="Junior Internal Auditor",
    goal="You are a Junior Internal Auditor, who is auditing vendors."
         "You have been provided the following {topic}"
         "your objective is to conduct a comprehensive evaluation of the issues"
         "observed by human auditor, The guideline for evaluation is:"
         "1. An issue statement describes a control breakdown that results in unacceptable risk"
         "2. An issue statement describes one of the following problems:"
         "a. The control task is not done (could be missing or circumvented)"
         "b. The control task is done, but in a flawed way"
         "3. An issue statement must be factually accurate (i.e., provable)"
         "4. An issue statement should be the first sentence in the Detailed Issue"
         "5. An issue statement should not describe a symptom instead of the control breakdown"
         "6. Issues are usually written in present tense"
         "7. Who does not [descriptive words] do what. [Active voice]"
         "8. What is not [descriptive words] done. [Passive voice]"
         "9. Avoid unnecessary introductions (e.g., During our audit; GA&A identified; It is observed that)."
         "This applies to all your writing, not just to issues"
         "10. Avoid starting with there, there is no, or no (e.g., There is no approval of…)."
         "There is not tangible and no has an attacking tone"
         "11. Avoid ensure, which means guarantee or make certain. Ensure is not a control"
         "12. Avoid recommendation words (e.g., should, needs to, requires)"
         "13. If needed for accuracy, add descriptive words around the verb. Consider flaws in frequency"
         "(e.g., not always, not regularly, not usually) and flaws in quality (e.g., not thoroughly, not consistently)"
         "14. Use present tense (e.g., does not, are not). Until the auditee corrects the issue and GA&A"
         "validates the correction, the residual risk remains unacceptable"
         "15. Write a complete sentence"
         "16. Avoid the phrase segregation of duties in the issue. It does not describe the control breakdown."
         "Instead, use conflicting control verbs or conflicting subjects."
         "You can highlight the conflict by using the word same or both."
         "Consider using “Segregation of Duties” as the title"
         "17. For full scope audits, avoid issues that focus on the following control elements or"
         "benchmarks: control, method, procedure, framework, policy, process. These issues are not task"
         "specific, might be misinterpreted, and often mask a more serious control breakdown"
         "18. When writing a consolidated issue, you still need to describe who does not do what or what is not done",
    backstory="You're working as a Junior Internal Auditor whose job is to audit vendors"
              "You always give output without violating the guidelines given to you",
    allow_delegation=False,
    verbose=True
)

SeniorLegalExpert = Agent(
    llm=llm,
    role="Senior Internal Auditor",

    goal=("you have to correct the issue {topic}:"
    "against the violated guidelines listed by Junior Internal Auditor."
    "Your task is to rewrite the issue statement so it does not have the"
    "guidelines violations as listed by Junior Internal Auditor"
    "Strictly consider to the data provided by human auditor as an issue"
    "statement and refrain from using external knowledge base"
    "Please rewrite the issue statement in which you have to include those"
    "points as it is, which are according to the guidelines and you have to"
    "correct only those points where violations listed by Junior Internal Auditor."
    "You should avoid repetition of information and be concise."
    "Do not write 'Rewritten Issue Statement' or 'Revised Issue Statement' or"
    "some similar keywords in the beginning of the corrected issue statement,"
    "just start the sentence without writing it."),

    backstory=("You are a Senior Internal Auditor, who is auditing vendors,"
              "you have to correct the issue statement :"
              "Processors of the off-platform VMD changes do not obtain reapproval"
              "from JnJ before processing request forms with critical post-approval"
              "modifications. Specifically, GAA identified that JnJ did not approve"
              "all eight samples of critical post-approval changes (e.g., changes"
              "in supporting documents, vendor names, and bank account names)"
              "processed by Wipro. GAA verified that all samples with"
              "post-changes were supported by valid documents"
              "against the violated guidelines listed by Junior Internal Auditor."
              "Your task is to rewrite the issue statement so it does not have the"
              "guidelines violations as listed by Junior Internal Auditor"
              "Strictly consider to the data provided by human auditor as an issue"
              "statement and refrain from using external knowledge base"),
    allow_delegation=False,
    verbose=True
)

ChiefLegalExpert = Agent(
    llm=llm,
    role="Chief Internal Auditor.",
    goal=("Your task is to validate the rewritten issue statement by"
    "Senior Internal Auditor based on guidelines. If all the violated"
    "guidelines have been corrected in the issue statement then confirm"
    "otherwise ask the Senior Internal Auditor to correct and rewrite the"
    "issue statement by adhering to the guidelines provided."
    "Never give the blank output, retrieve all the previous context and"
    "discussion and provide the answer despite of blank answer"
    "or never give update context as final answer"
    "Do not write 'Rewritten Issue Statement' or 'Revised Issue Statement'"
    "or some similar keywords in the beginning of the corrected issue statement,"
    "just start the sentence without writing it."
    "You should avoid repetition of information and be concise."),
    backstory=("You are a Cheif Internal Auditor who check the work submitted by Senior Internal Auditor"
    "Your task is to validate the rewritten issue statement by"
    "Senior Internal Auditor based on guidelines."),
    allow_delegation=False,
    verbose=True
)

In [5]:
Audit = Task(
    description=(
        "conduct a comprehensive evaluation of the issue {topic}"),
    expected_output=
    "observed by human auditor, The guideline for evaluation is:"
    "1. An issue statement describes a control breakdown that results in unacceptable risk"
    "2. An issue statement describes one of the following problems:"
    "a. The control task is not done (could be missing or circumvented)"
    "b. The control task is done, but in a flawed way"
    "3. An issue statement must be factually accurate (i.e., provable)"
    "4. An issue statement should be the first sentence in the Detailed Issue"
    "5. An issue statement should not describe a symptom instead of the control breakdown"
    "6. Issues are usually written in present tense"
    "7. Who does not [descriptive words] do what. [Active voice]"
    "8. What is not [descriptive words] done. [Passive voice]"
    "9. Avoid unnecessary introductions (e.g., During our audit; GA&A identified; It is observed that)."
        "This applies to all your writing, not just to issues"
        "10. Avoid starting with there, there is no, or no (e.g., There is no approval of…)."
        "There is not tangible and no has an attacking tone"
        "11. Avoid ensure, which means guarantee or make certain. Ensure is not a control"
        "12. Avoid recommendation words (e.g., should, needs to, requires)"
        "13. If needed for accuracy, add descriptive words around the verb. Consider flaws in frequency"
        "(e.g., not always, not regularly, not usually) and flaws in quality (e.g., not thoroughly, not consistently)"
        "14. Use present tense (e.g., does not, are not). Until the auditee corrects the issue and GA&A"
        "validates the correction, the residual risk remains unacceptable"
        "15. Write a complete sentence"
        "16. Avoid the phrase segregation of duties in the issue. It does not describe the control breakdown."
        "Instead, use conflicting control verbs or conflicting subjects."
        "You can highlight the conflict by using the word same or both."
        "Consider using “Segregation of Duties” as the title"
        "17. For full scope audits, avoid issues that focus on the following control elements or"
        "benchmarks: control, method, procedure, framework, policy, process. These issues are not task"
        "specific, might be misinterpreted, and often mask a more serious control breakdown"
        "18. When writing a consolidated issue, you still need to describe who does not do what or what is not done",
    agent=JuniorLegalExpert,
)

Correction = Task(
    description=("you have to correct the issue {topic}"
    "against the violated guidelines listed by Junior Internal Auditor."
    "Your task is to rewrite the issue statement so it does not have the"
    "guidelines violations as listed by Junior Internal Auditor"
    "Strictly consider to the data provided by human auditor as an issue"
    "statement and refrain from using external knowledge base"
    "Please rewrite the issue statement in which you have to include those"
    "points as it is, which are according to the guidelines and you have to"
    "correct only those points where violations listed by Junior Internal Auditor."
    "You should avoid repetition of information and be concise."),
    expected_output=
              "rewrite the issue statement so it does not have the"
              "guidelines violations as listed by Junior Internal Auditor"
              "Strictly consider to the data provided by human auditor as an issue"
              "statement and refrain from using external knowledge base"
              "Do not write 'Rewritten Issue Statement' or 'Revised Issue Statement' or"
              "some similar keywords in the beginning of the corrected issue statement,"
              "just start the sentence without writing it.",
    agent=SeniorLegalExpert,
)

Rewrite = Task(
    description=(
        "Your task is to validate the rewritten issue statement by"
        "Senior Internal Auditor based on guidelines. If all the violated"
        "guidelines have been corrected in the issue statement then confirm"
        "otherwise ask the Senior Internal Auditor to correct and rewrite the"
        "issue statement by adhering to the guidelines provided."
        ),
    expected_output=
    "Never give blank output, retrieve all the previous context and"
    "discussion and provide the answer despite of blank answer"
    "or never give update context as final answer"
    "Do not write 'Rewritten Issue Statement' or 'Revised Issue Statement'"
    "or some similar keywords in the beginning of the corrected issue statement,"
    "just start the sentence without writing it."
    "You should avoid repetition of information and be concise",
    agent=ChiefLegalExpert
)

In [6]:
crew = Crew(
    agents=[JuniorLegalExpert, SeniorLegalExpert, ChiefLegalExpert],
    tasks=[Audit, Correction, Rewrite],
    verbose=2
)

In [9]:
result = crew.kickoff(inputs={"topic": "Processors of the off-platform VMD changes do not obtain reapproval"
         "from JnJ before processing request forms with critical post-approval"
         "modifications. Specifically, GAA identified that JnJ did not approve"
         "all eight samples of critical post-approval changes (e.g., changes"
         "in supporting documents, vendor names, and bank account names)"
         "processed by Wipro. GAA verified that all samples with"
         "post-changes were supported by valid documents"})

 [DEBUG]: == Working Agent: Junior Internal Auditor
 [INFO]: == Starting Task: conduct a comprehensive evaluation of the issue Processors of the off-platform VMD changes do not obtain reapprovalfrom JnJ before processing request forms with critical post-approvalmodifications. Specifically, GAA identified that JnJ did not approveall eight samples of critical post-approval changes (e.g., changesin supporting documents, vendor names, and bank account names)processed by Wipro. GAA verified that all samples withpost-changes were supported by valid documents


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: Wipro does not obtain reapproval from JnJ before processing request forms with critical post-approval modifications.

> Finished chain.
 [DEBUG]: == [Junior Internal Auditor] Task output: Wipro does not obtain reapproval from JnJ before processing request forms with critical post-approval modifications.


 [DEBUG]: == Working Agent: Senior I

In [10]:
crew.usage_metrics

{'total_tokens': 65250,
 'prompt_tokens': 60894,
 'completion_tokens': 4356,
 'successful_requests': 63}